In [4]:
from nba_api.stats.static import teams, players
from nba_api.stats.endpoints import cumestatsteamgames, cumestatsteam, gamerotation
import pandas as pd
import numpy as np
import json
import difflib
import time
import requests
import re

In [11]:
from nba_api.stats.static import teams as nba_teams_module
from nba_api.stats.endpoints.commonteamroster import CommonTeamRoster
from nba_api.stats.endpoints.leaguestandingsv3 import LeagueStandingsV3

# Create Team and Roster Class

In [32]:
class NBATeamRosters:
    def __init__(self, season):
        """
        Initializes the class and sets up the teams DataFrame and an empty dictionary for rosters.
        Args:
            season (str): The NBA season in 'YYYY' format, e.g., '2024' for the 2024-2025 season.
        """
        self.season = season
        self.teams_df = None
        self.rosters = {}
        self.standings_df = None
        self.schedule_df = None
        # Fetch teams and prepare the teams DataFrame
        self.fetch_teams()

    def fetch_teams(self):
        """
        Fetches NBA teams and stores them in a DataFrame.
        """
        nba_teams = nba_teams_module.get_teams()
        self.teams_df = pd.DataFrame(nba_teams).set_index('id')

    def fetch_rosters(self):
        """
        Fetches and stores the rosters for each team.
        The rosters are stored in a dictionary with keys as the team's abbreviation, nickname, and city.
        """
        for team_id, team_data in self.teams_df.iterrows():
            team_roster = self.get_team_roster(team_id)
            if team_roster is not None:
                self.rosters[team_data['abbreviation']] = team_roster
                self.rosters[team_data['nickname']] = team_roster
                self.rosters[team_data['city']] = team_roster

    def get_team_roster(self, team_id):
        """
        Fetches the roster for a single team based on the team ID.
        Args:
            team_id (int): The team's ID.

        Returns:
            pd.DataFrame: DataFrame containing the team's roster.
        """
        try:
            rosters_raw = CommonTeamRoster(
                team_id=str(team_id),
                season=self.season
            ).common_team_roster.get_data_frame()
            return rosters_raw
        except Exception as e:
            print(f"Error fetching roster for team ID {team_id}: {e}")
            return None

    def get_roster_by_key(self, key):
        """
        Retrieves the roster DataFrame by the team's abbreviation, nickname, or city.
        Args:
            key (str): The team's abbreviation, nickname, or city.

        Returns:
            pd.DataFrame: DataFrame containing the team's roster.
        """
        return self.rosters.get(key, None)

    def load_schedule(self, csv_path):
        """
        Loads the NBA schedule from a CSV file into the class.
        Args:
            csv_path (str): The file path to the schedule CSV file.
        """
        self.schedule_df = pd.read_csv(csv_path)
        print(f"Schedule loaded: {len(self.schedule_df)} games")

    def get_teams_for_game(self, game_id):
        """
        Retrieves the teams playing in a specific game from the schedule.
        Args:
            game_id (str): The ID of the game.

        Returns:
            Tuple[pd.DataFrame, pd.DataFrame]: DataFrames for the two teams playing the game.
        """
        game = self.schedule_df.loc[self.schedule_df['Game ID'] == game_id].iloc[0]
        home_team = self.teams_df[self.teams_df['abbreviation'] == game['Home Team Abbreviation']]
        away_team = self.teams_df[self.teams_df['abbreviation'] == game['Visiting Team Abbreviation']]
        return home_team, away_team

    def get_todays_games(self, today):
        """
        Retrieves the games scheduled for a specific date.
        Args:
            today (str): The date in 'YYYY-MM-DD' format.

        Returns:
            pd.DataFrame: DataFrame containing the games scheduled for the specified date.
        """
        if self.schedule_df is not None:
            todays_games = self.schedule_df[self.schedule_df["Game Date"] == today]
            return todays_games
        else:
            print("Schedule not loaded.")
            return None

    def get_full_schedule(self):
        """
        Returns the full schedule DataFrame.
        Returns:
            pd.DataFrame: DataFrame containing the entire schedule.
        """
        if self.schedule_df is not None:
            return self.schedule_df
        else:
            print("Schedule not loaded.")
            return None
    
    def fetch_league_standings(self, season_type):
        """
        Fetches the league standings for the specified season and season type.
        Args:
            season_type (str): The type of season ('Regular Season', 'Playoffs', etc.)

        Returns:
            pd.DataFrame: DataFrame containing the league standings.
        """
        league_standings = LeagueStandingsV3(
            league_id="00",
            season=self.season,
            season_type=season_type
        )
        standings_data = league_standings.standings.get_dict()
        self.standings_df = pd.DataFrame(standings_data['data'], columns=standings_data['headers'])
        self.standings_df.index = np.arange(1, len(self.standings_df) + 1)
        self.standings_df = self.standings_df.drop(columns=['SeasonID', 'LeagueID', 'TeamSlug', 'LeagueRank'])
        self.standings_df = self.standings_df.rename(columns={'TeamID': 'id'})
        return self.standings_df

    def process_league_standings(self, season_type):
        """
        Processes the league standings by division and conference.
        Args:
            season_type (str): The type of season ('Regular Season', 'Playoffs', etc.)

        Returns:
            pd.DataFrame: DataFrame sorted by Conference and Division.
        """
        # Fetch and process the standings
        league_standings = self.fetch_league_standings(season_type)

        # Group by 'Division' and create separate DataFrames for each division
        grouped = league_standings.groupby('Division')
        divisions = {division_name: group for division_name, group in grouped}
        
        # Prepare and modify DataFrames for each division
        modified_dfs = []
        for division, df in divisions.items():
            # Create a separator row with the division name
            separator_df = pd.DataFrame([{col: '' for col in df.columns}])
            separator_df.iloc[0, df.columns.get_loc('Division')] = division
            
            # Concatenate the separator row and the division DataFrame
            combined_df = pd.concat([separator_df, df])
            
            # Append the combined DataFrame to the list
            modified_dfs.append(combined_df)
        
        # Combine all modified DataFrames
        final_df = pd.concat(modified_dfs, ignore_index=True)
        
        # Order by 'Conference' and set specific values
        conference_order = ['East', 'West']
        final_df['Conference'] = pd.Categorical(final_df['Conference'], categories=conference_order, ordered=True)
        
        # Set specific 'Conference' values based on the provided indices
        final_df.loc[0, 'Conference'] = "East"
        final_df.loc[6, 'Conference'] = "East"
        final_df.loc[12, 'Conference'] = "West"
        final_df.loc[18, 'Conference'] = "West"
        final_df.loc[24, 'Conference'] = "East"
        final_df.loc[30, 'Conference'] = "West"
        
        # Sort the DataFrame by 'Conference' and 'Division'
        final_df_sorted = final_df.sort_values(by=['Conference', 'Division'])
        
        return final_df_sorted

# Roster Search

In [20]:
# Example usage
nba_rosters = NBATeamRosters(season="2024")
nba_rosters.fetch_rosters()

In [ ]:
# Fetch and print the teams DataFrame
teams_df = nba_data.teams_df
teams_df

In [21]:
# Retrieve roster by abbreviation, nickname, or city
lakers_roster = nba_rosters.get_roster_by_key('LAL')
lakers_roster

In [29]:
celtics_roster = nba_rosters.get_roster_by_key('Celtics')
celtics_roster

,TeamID,SEASON,LeagueID,PLAYER,NICKNAME,PLAYER_SLUG,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL,PLAYER_ID,HOW_ACQUIRED
0,1610612738,2024,00,Baylor Scheierman,Baylor,baylor-scheierman,None,G,6-6,205,"SEP 26, 2000",23.0,R,Creighton,1631248,#30 Pick in 2024 Draft
1,1610612738,2024,00,Anton Watson,Anton,anton-watson,None,F,6-8,225,"OCT 06, 2000",23.0,R,Gonzaga,1641817,#54 Pick in 2024 Draft
2,1610612738,2024,00,Tristan Enaruna,Tristan,tristan-enaruna,None,F,6-8,220,"JUN 26, 2001",23.0,R,Cleveland State,1642400,Signed on 07/10/24
3,1610612738,2024,00,Jayson Tatum,Jayson,jayson-tatum,0,F-G,6-8,210,"MAR 03, 1998",26.0,7,Duke,1628369,#3 Pick in 2017 Draft
4,1610612738,2024,00,Jrue Holiday,Jrue,jrue-holiday,4,G,6-4,205,"JUN 12, 1990",34.0,15,UCLA,201950,Traded from POR on 10/01/23
5,1610612738,2024,00,Jaylen Brown,Jaylen,jaylen-brown,7,G-F,6-6,223,"OCT 24, 1996",27.0,8,California,1627759,#3 Pick in 2016 Draft
6,1610612738,2024,00,Kristaps Porzingis,Kristaps,kristaps-porzingis,8,F-C,7-2,240,"AUG 02, 1995",29.0,8,Cajasol Sevilla,204001,Traded from WAS on 06/23/23
7,1610612738,2024,00,Ron Harper Jr.,Ron,ron-harper-jr,8,G-F,6-4,233,"APR 12, 2000",24.0,2,Rutgers,1631199,Signed on 07/13/24
8,1610612738,2024,00,Lonnie Walker IV,Lonnie,lonnie-walker-iv,8,G-F,6-4,204,"DEC 14, 1998",25.0,6,Miami,1629022,Signed on 08/28/24
9,1610612738,2024,00,Derrick White,Derrick,derrick-white,9,G,6-4,190,"JUL 02, 1994",30.0,7,Colorado,1628401,Traded from SAS on 02/10/22


In [ ]:
boston_roster = nba_rosters.get_roster_by_key('Boston')
boston_roster

# Standings

In [33]:
# Instantiate the class for the 2024-2025 season
nba_data = NBATeamRosters(season="2024")

In [34]:
# Fetch and process league standings for the Regular Season
processed_standings = nba_data.process_league_standings(season_type="Regular Season")
processed_standings

,id,TeamCity,TeamName,Conference,ConferenceRecord,PlayoffRank,ClinchIndicator,Division,DivisionRecord,DivisionRank,...,Score_80_Plus,Opp_Score_80_Plus,Score_Below_80,Opp_Score_Below_80,TotalPoints,OppTotalPoints,DiffTotalPoints,LeagueGamesBack,PlayoffSeeding,ClinchedPostSeason
0,,,,East,,,,Atlantic,,,...,,,,,,,,,,
1,1610612738,Boston,Celtics,East,0-0,2,,Atlantic,0-0,1,...,0-0,0-0,0-0,0-0,0,0,0,0.0,None,0
2,1610612751,Brooklyn,Nets,East,0-0,4,,Atlantic,0-0,2,...,0-0,0-0,0-0,0-0,0,0,0,0.0,None,0
3,1610612752,New York,Knicks,East,0-0,11,,Atlantic,0-0,3,...,0-0,0-0,0-0,0-0,0,0,0,0.0,None,0
4,1610612755,Philadelphia,76ers,East,0-0,13,,Atlantic,0-0,4,...,0-0,0-0,0-0,0-0,0,0,0,0.0,None,0
5,1610612761,Toronto,Raptors,East,0-0,14,,Atlantic,0-0,5,...,0-0,0-0,0-0,0-0,0,0,0,0.0,None,0
6,,,,East,,,,Central,,,...,,,,,,,,,,
7,1610612741,Chicago,Bulls,East,0-0,3,,Central,0-0,1,...,0-0,0-0,0-0,0-0,0,0,0,0.0,None,0
8,1610612739,Cleveland,Cavaliers,East,0-0,6,,Central,0-0,2,...,0-0,0-0,0-0,0-0,0,0,0,0.0,None,0
9,1610612765,Detroit,Pistons,East,0-0,7,,Central,0-0,3,...,0-0,0-0,0-0,0-0,0,0,0,0.0,None,0


# Schedule

In [35]:
# Load the schedule from a CSV file
nba_data.load_schedule("nbaSchedule24.csv")

Schedule loaded: 1200 games


In [38]:
todays_games = nba_data.get_todays_games('2024-10-22')
todays_games

,Game Date,Game ID,Game Time,Arena,Arena City,Home Team Abbreviation,Home Conference,Home Division,Visiting Team Abbreviation,Visiting Conference,Visiting Division,Divisional Game,Conference Game
0,2024-10-22,22400061,7:30 pm ET,TD Garden,Boston,BOS,East,Atlantic,NYK,East,Atlantic,Yes,Yes
1,2024-10-22,22400062,10:00 pm ET,Crypto.com Arena,Los Angeles,LAL,West,Pacific,MIN,West,Northwest,No,Yes


In [39]:
# Get the full schedule
full_schedule = nba_data.get_full_schedule()
full_schedule

,Game Date,Game ID,Game Time,Arena,Arena City,Home Team Abbreviation,Home Conference,Home Division,Visiting Team Abbreviation,Visiting Conference,Visiting Division,Divisional Game,Conference Game
0,2024-10-22,22400061,7:30 pm ET,TD Garden,Boston,BOS,East,Atlantic,NYK,East,Atlantic,Yes,Yes
1,2024-10-22,22400062,10:00 pm ET,Crypto.com Arena,Los Angeles,LAL,West,Pacific,MIN,West,Northwest,No,Yes
2,2024-10-23,22400067,7:30 pm ET,Scotiabank Arena,Toronto,TOR,East,Atlantic,CLE,East,Central,No,Yes
3,2024-10-23,22400072,10:00 pm ET,Moda Center,Portland,POR,West,Northwest,GSW,West,Pacific,No,Yes
4,2024-10-23,22400071,10:00 pm ET,Intuit Dome,Inglewood,LAC,West,Pacific,PHX,West,Pacific,Yes,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,2025-04-13,22401189,1:00 pm ET,Rocket Mortgage FieldHouse,Cleveland,CLE,East,Central,IND,East,Central,Yes,Yes
1196,2025-04-13,22401188,1:00 pm ET,Barclays Center,Brooklyn,BKN,East,Atlantic,NYK,East,Atlantic,Yes,Yes
1197,2025-04-13,22401187,1:00 pm ET,TD Garden,Boston,BOS,East,Atlantic,CHA,East,Southeast,No,Yes
1198,2025-04-13,22401194,3:30 pm ET,FedExForum,Memphis,MEM,West,Southwest,DAL,West,Southwest,Yes,Yes
